In [26]:
from pathlib import Path
import requests 

##  Function to extract raw parquet data from the NYC taxi data 

In [27]:
def download_one_file_of_raw_data(year: int, month: int) -> Path:
    """"""
    URL = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet"
    response = requests.get(URL)

    if response.status_code == 200:
        path = f"../data/raw/rides_{year}-{month:02d}.parquet"
        open(path, "wb").write(response.content)
        return path
    else:
        raise Exception(f"{URL} is not available")

In [28]:
download_one_file_of_raw_data(year=2022, month=1) 

'../data/raw/rides_2022-01.parquet'

In [29]:
import pandas as pd 

rides = pd.read_parquet("../data/raw/rides_2022-01.parquet")

In [30]:
rides.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.00,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.00,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.00,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.00,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.00,0.3,30.30,2.5,0.0
5,1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.30,1.0,N,138,161,1,33.0,3.0,0.5,13.00,6.55,0.3,56.35,2.5,0.0
6,2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,N,233,87,1,17.0,0.5,0.5,5.20,0.00,0.3,26.00,2.5,0.0
7,2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,N,238,152,2,9.0,0.5,0.5,0.00,0.00,0.3,12.80,2.5,0.0
8,2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,N,166,236,1,12.0,0.5,0.5,2.25,0.00,0.3,18.05,2.5,0.0
9,2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,N,236,141,2,5.0,0.5,0.5,0.00,0.00,0.3,8.80,2.5,0.0


### Make the rides dataframe only include the 2 columns pickup date time and pickup location id

In [32]:
rides = rides[["tpep_pickup_datetime", "PULocationID"]]

Rename the columns

In [33]:
rides.rename(columns={
    "tpep_pickup_datetime": "pickup_datetime",
    "PULocationID":"pickup_location_id"
    }, inplace = True)

rides.head(10)

/var/folders/rz/v2_v6xwd24v6fdtmrs313wmr0000gn/T/ipykernel_1018/4232121849.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rides.rename(columns={


,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
5,2022-01-01 00:40:15,138
6,2022-01-01 00:20:50,233
7,2022-01-01 00:13:04,238
8,2022-01-01 00:30:02,166
9,2022-01-01 00:48:52,236


### Here we see that there is a date from 2008 and a date from 2022 may, although this data is only supposed to be from jan 2022. let remove it...

In [34]:

rides["pickup_datetime"].describe()

/var/folders/rz/v2_v6xwd24v6fdtmrs313wmr0000gn/T/ipykernel_1018/1148853342.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides["pickup_datetime"].describe()


count                 2463931
unique                1423522
top       2022-01-26 07:57:00
freq                       12
first     2008-12-31 22:23:09
last      2022-05-18 20:41:57
Name: pickup_datetime, dtype: object

### Making sure the data actually only consists of 2022 January data 

In [35]:
rides = rides[rides.pickup_datetime >= "2022-01-01"]
rides = rides[rides.pickup_datetime < "2022-02-01"]
rides["pickup_datetime"].describe()

/var/folders/rz/v2_v6xwd24v6fdtmrs313wmr0000gn/T/ipykernel_1018/2894541148.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  rides["pickup_datetime"].describe()


count                 2463879
unique                1423471
top       2022-01-26 07:57:00
freq                       12
first     2022-01-01 00:00:08
last      2022-01-31 23:59:58
Name: pickup_datetime, dtype: object

### Store transformed file back to disk

In [36]:
rides.to_parquet("../data/transformed/validated_rides_2022_01.parquet")